In [26]:
import cv2
import numpy as np
import pandas as pd

from collections import Counter

from keras.callbacks import Callback
from keras.backend import clear_session
from keras.models import Model, load_model
from keras.layers import Dense, Input, Flatten
from keras.applications import ResNet50, MobileNet, Xception, DenseNet121

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa

In [25]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import pandas as pd
import cv2
from pathlib import Path
import os
import glob
import sys
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.keras.models import Sequential as seq
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.layers import AveragePooling2D
from keras_preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

# Data Loader
The Notebook has been executed on google colab so we mount the drive and use the data from the drive

In [6]:

test_path='/content/drive/MyDrive/test_images'
trainpath='/content/drive/MyDrive/train_images'
train_cpath='/content/drive/MyDrive/train.csv'
test_cpath='/content/drive/MyDrive/test.csv'
df = pd.read_csv(train_cpath)  
df_test = pd.read_csv(test_cpath)
print(df)
classes = list(np.unique(df['ClassName']))
print(len(classes))
classDict = {}
for i in range(61):
    classDict[classes[i]]=i
imgIdsList=df['ImageId']
clsNameList=df['ClassName']
trial_label={}
for i in range(len(df)):
    trial_label[imgIdsList[i]]=classDict[clsNameList[i]]
X = []
Y = []
l=0
for imgName,clsID in trial_label.items():
    currImagePath=os.path.join(trainpath,imgName)
    img=cv2.imread(currImagePath)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    resized=cv2.resize(img,(150,150))
    X.append(resized)
    Y.append(clsID)
    l+=1
    if(l%100==0):
      print('#',end='')
print(len(X))

             ImageId                 ClassName
0     f27632d7e5.jpg                     water
1     efa87919ed.jpg    pizza-margherita-baked
2     4f169e8c8d.jpg                  broccoli
3     a6956654bf.jpg    salad-leaf-salad-green
4     d99ce8c3bf.jpg                       egg
...              ...                       ...
9318  ba8233c7d2.jpg  bread-french-white-flour
9319  2090043907.jpg                  wine-red
9320  8762d1cefd.jpg                    carrot
9321  28e7439245.jpg                  broccoli
9322  ba263cfb41.jpg                  cucumber

[9323 rows x 2 columns]
61
#############################################################################################9323


In [29]:
print(len(Y))

9323


In [8]:
X=np.asarray(X,'float')
print(X.shape)
Y_array=np.array(Y)
X_train,X_val,y_train,y_val=train_test_split(X,Y_array,test_size=0.30,random_state=2)
print(len(y_train))
## Normalization

X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

MEAN = np.mean(X_train, axis = (0,1,2))
STD = np.std(X_train, axis = (0,1,2))

#for i in range(3):
 #   X_train[:, :, :, i] = (X_train[:, :, :, i] - MEAN[i]) / STD[i]
  #  X_val[:, :, :, i] = (X_val[:, :, :, i] - MEAN[i]) / STD[i]

(9323, 150, 150, 3)
6526


In [36]:
early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

# Model
The models experimented are DenseNet121,MobileNet, Xception or ResNet50. Xception outperformed in this problem's case

In [31]:
clear_session()
img = Input(shape = (150, 150, 3))

## Tried DenseNet121 , MobileNet, Xception & ResNet50
model = Xception(include_top=True, 
                    weights='imagenet', 
                    input_tensor=img, 
                    input_shape=None, 
                    pooling='max')
final_layer = model.layers[-1].output

dense_layer_1 = Dense(128, activation = 'relu')(final_layer)
output_layer = Dense(61, activation = 'softmax')(dense_layer_1)

model = Model(img,output_layer)

91889664/91884032 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = (X_val, y_val),callbacks=[early_stopping_monitor])

Epoch 1/10
 60/204 [=======>......................] - ETA: 40s - loss: 3.1459 - accuracy: 0.2077

The test data is loaded and the model is used to predict the labesls and the labels are converted using the class dictionary obtained earlier from the training set

In [ ]:
print(df_test)
imgIdsList=df_test['ImageId']
X_test = []
l=0
for imgName in imgIdsList:
    currImagePath=os.path.join(test_path,imgName)
    img=cv2.imread(currImagePath)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    resized=cv2.resize(img,(150,150))
    X_test.append(resized)
    l+=1


In [13]:
X_test=np.asarray(X_test,'float')
#for i in range(3):
  #X_test[:, :, :, i] = (X_test[:, :, :, i] - MEAN[i]) / STD[i]
for i in range(61):
    classDict[classes[i]]=i

In [14]:
X_test=np.asarray(X_test,'float')
y_test=model.predict(X_test)
def get_key(val):
    for key, value in classDict.items():
         if val == value:
             return key
 
    return "key doesn't exist"
test=np.argmax(y_test,axis=1)
test2=[]
for example in test:
    test2.append(get_key(example))
df = pd.DataFrame(test2) 
df.columns=['ClassName']    
# saving the dataframe 
df.to_csv('GFG.csv',index=False) 

The ClassNames for the 483 images are as shown

In [15]:
df

,ClassName
0,water
1,water
2,coffee-with-caffeine
3,hard-cheese
4,bread-wholemeal
...,...
479,broccoli
480,pasta-spaghetti
481,water
482,bread-wholemeal
